In [2]:
import qiskit as qk
from qiskit.circuit import Parameter
import numpy as np
from qiskit_algorithms import TimeEvolutionProblem , TrotterQRTE
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import EfficientSU2
from qiskit_aer import Aer
from qiskit import transpile

In [3]:

# Define the system parameters
N = 2        # Number of qubits
h = 1.0      # Coefficient for the Z_i terms
V = 1.0      # Coefficient for the Z_i Z_{i+1} terms
c0 = 1.0     # Coefficient for the time-dependent X_i X_{i+1} terms
omega = 1.0  # Frequency for the cosine term
cs = [2.0 + (-1)**x for x in range(N-1)]
period = 2*np.pi/omega
total_time = 2*period

In [6]:
Trotter_circ = qk.QuantumCircuit(N)

time_step = 1.0 / omega 
num_trotter_steps = int( total_time / time_step )

t_value = time_step
cos_factor = c0 * np.cos(omega * t_value)

for _ in range(num_trotter_steps):
    # Z terms
    for i in range(N):
        Trotter_circ.rz(2 * h * time_step , i)
    
    # ZZ terms
    for i in range(N - 1):
        Trotter_circ.rzz(2 * V * time_step , i, i + 1)
    
    # XX terms with the time-dependent cosine factor
    for i in range(N - 1):
        Trotter_circ.rxx(2 * cos_factor * time_step , i, i + 1)

simulator = Aer.get_backend('aer_simulator')
transpiled_circuit = transpile(Trotter_circ, simulator)



In [7]:
print(transpiled_circuit.draw())

     ┌───────┐                ┌──────────────┐   ┌───────┐                    »
q_0: ┤ Rz(2) ├─■──────────────┤0             ├───┤ Rz(2) ├────────────────────»
     ├───────┤ │ZZ(2)         │  Rxx(1.0806) │┌──┴───────┴───┐   ┌───────┐    »
q_1: ┤ Rz(2) ├─■───────■──────┤1             ├┤0             ├───┤ Rz(2) ├────»
     ├───────┤         │ZZ(2) └──────────────┘│  Rxx(1.0806) │┌──┴───────┴───┐»
q_2: ┤ Rz(2) ├─────────■───────────■──────────┤1             ├┤0             ├»
     ├───────┤                     │ZZ(2)     └──────────────┘│  Rxx(1.0806) │»
q_3: ┤ Rz(2) ├─────────────────────■──────────────────────────┤1             ├»
     └───────┘                                                └──────────────┘»
«                      ┌──────────────┐   ┌───────┐                    »
«q_0: ──■──────────────┤0             ├───┤ Rz(2) ├────────────────────»
«       │ZZ(2)         │  Rxx(1.0806) │┌──┴───────┴───┐   ┌───────┐    »
«q_1: ──■───────■──────┤1             ├┤0             ├───┤ R